In [1]:
from model import PM10CNN
from dataset import PM10Dataset
from utils import imputed_china_pm_data

df = imputed_china_pm_data()
dataset = PM10Dataset(df)

dataset

/home/sangwon/nlplab_leaderboard2/Junho/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)


In [2]:
print(dataset[0].shape)
print(dataset[0])

torch.Size([56, 4])
tensor([[195.0000,  54.0000,  14.0000,  49.0000],
        [131.0000,  73.0000,   7.0000,  94.0000],
        [151.0000,  93.0000,  11.0000,  71.0000],
        [171.0000, 104.0000,  73.0000, 120.0000],
        [129.5000, 118.0000,  11.0000,  30.0000],
        [ 88.0000,  84.0000,  17.0000,  43.0000],
        [178.0000,  65.0000,  49.0000,  47.0000],
        [113.0000,  38.0000,  29.0000,  33.0000],
        [131.0000,  27.0000,  46.0000,  25.5000],
        [127.0000,  29.0000,  26.0000,  18.0000],
        [146.0000,  27.0000,  23.0000,  40.0000],
        [141.0000,  81.0000,  44.0000,  32.0000],
        [ 40.0000,  61.0000,  29.0000,  63.0000],
        [ 40.0000, 151.0000,  30.0000,  50.0000],
        [ 42.0000,  69.0000,  31.0000,  18.0000],
        [ 39.0000,  52.0000,  18.0000,  64.0000],
        [ 19.0000,  26.0000,  12.0000,  53.0000],
        [ 38.0000,  34.0000,  10.0000,  37.0000],
        [ 50.0000,  47.0000,  22.0000,  41.0000],
        [106.0000,  53.0000,  

In [4]:
model = PM10CNN(in_channels=4, out_channels=16)

out = model(dataset[0].unsqueeze(0))
print(out.shape)

torch.Size([1, 56, 16])


In [5]:
model = PM10CNN(in_channels=4, out_channels=16, K=3)

out = model(dataset[0].unsqueeze(0))
print(out.shape)

torch.Size([1, 56, 48])


* 테스트 성공!

## Full model test

In [1]:
from tqdm import tqdm
from utils import set_logger, prepare_data_for_CNN_LSTM
from dataset import FinedustCNNLSTMDataset
from model import FinedustCNNLSTM
from utils import prepare_data
from sklearn.model_selection import train_test_split
import os
import logging
from interpolate import simple_interpolate
import numpy as np
from preprocess import minmax_scaling
from train import train, train_cnn_lstm
from permutation import compute_cnn_lstm_permutation_importance

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
config = {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "window_size": 24,
    "output_size": 1,
    "dropout_prob": 0.2,
    "patience": 10,
    "out_channels": 16,
    "kernel_size": 3,
    "K": 3,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from dataset import FinedustCNNLSTMDataset
from model import FinedustCNNLSTM
from utils import prepare_data_for_CNN_LSTM
from interpolate import simple_interpolate
from preprocess import minmax_scaling

region = "Seoul"
if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]

df = prepare_data_for_CNN_LSTM(region.lower(), columns_to_remove=columns_to_remove)
df = simple_interpolate(df, method="linear")
df, scaler = minmax_scaling(df)

dataset = FinedustCNNLSTMDataset(df,
                                window_size=config["window_size"],
                                prediction_length=config["output_size"],
                                time_window=3)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

/home/han07301/ai_project/JH/CNN_LSTM/utils.py:119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[pm_columns] = df[pm_columns].applymap(lambda x: np.nan if x <= 0 else x)
/home/han07301/ai_project/JH/CNN_LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')


In [4]:
model = FinedustCNNLSTM(config,
                        in_channels=len(dataset.pm_columns),
                        input_size=len(dataset.feature_columns)).to(device)

In [5]:
print(model)

FinedustCNNLSTM(
  (cnn): PM10CNN(
    (act): GELU(approximate='none')
    (conv_list): ModuleList(
      (0): Conv1d(4, 16, kernel_size=(3,), stride=(1,), padding=(1,))
      (1-2): 2 x Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    )
  )
  (lstm): FinedustLSTM(
    (lstm): LSTMEmbedding(
      (lstm): LSTM(60, 128, num_layers=2, batch_first=True, dropout=0.2)
    )
    (fc1): Linear(in_features=128, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=1, bias=True)
    (act): GELU(approximate='none')
    (dropout): Dropout(p=0.2, inplace=False)
  )
)


In [6]:
from model import PM10CNN

batch = next(iter(train_loader))
w_x, pm_x, y = batch
print(w_x.shape, pm_x.shape, y.shape)

w_x, pm_x, y = w_x.to(device), pm_x.to(device), y.to(device)
out = model(w_x, pm_x)

# pm_cnn = PM10CNN(in_channels=4, out_channels=16, kernel_size=3, K=3)
# pm_out = pm_cnn(pm_x)
# print(pm_out.shape)

torch.Size([32, 24, 12]) torch.Size([32, 24, 4]) torch.Size([32, 1])
torch.Size([32, 24, 12]) torch.Size([32, 24, 4])
torch.Size([32, 24, 48])
torch.Size([32, 24, 60])
torch.Size([32, 1])


In [10]:
torch.cat((w_x, pm_out), dim=-1).shape


torch.Size([32, 24, 60])